In [1]:
import math
import random
import numpy as np
from itertools import product

from qiskit import *

# Importing standard Qiskit libraries
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator

from qiskit.utils import QuantumInstance

from qiskit.opflow import I, X, Y, Z

from qiskit.opflow import StateFn
from qiskit.opflow import Gradient
from qiskit.opflow import CircuitSampler
from qiskit.opflow.primitive_ops import MatrixOp

from qiskit.quantum_info import Statevector

from qiskit.providers.aer.noise import NoiseModel
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
def createEqSuperposDihedral():
    '''
    Defines a quantum circuit to create the uniform superposition of basis elements required.
    '''
    temp = QuantumCircuit(3)
    temp.ry(2*np.arctan(1/np.sqrt(2)), 0)
    temp.x(0)
    temp.ch(0, 1)
    temp.x(0)
    temp.h(2)
    return temp

In [4]:
'''
Defines a quantum circuit to create a purification of the state to be tested. Purification created on qubits (S, S') - 
two qubits each.
'''
size = 4
state = QuantumCircuit(size)
state.ry(2*np.arcsin(np.sqrt(2/3)), 0)
state.ch(0, 1)
state.x(1)
state.draw()

┌────────────┐          
q_0: ┤ Ry(1.9106) ├──■───────
     └────────────┘┌─┴─┐┌───┐
q_1: ──────────────┤ H ├┤ X ├
                   └───┘└───┘
q_2: ────────────────────────
                             
q_3: ────────────────────────

In [5]:
def createQuantumCircuit():
    '''
    Creates quantum circuit for the problem. 
    Refer to Section 3.A from https://arxiv.org/pdf/2105.12758
    The ciruit contains 7 qubits each labelled as (C, S, S')
    '''
    circ = QuantumCircuit(7)
    #Create Quantum Circuit here
    
    superPosCircuit = createEqSuperposDihedral()
    circ.append(superPosCircuit, [0, 1, 2])
    circ.append(state, [3, 4, 5, 6])

    circ.ccx(2, 4, 3)
    circ.cswap(1, 3, 4)
    circ.ccx(0, 3, 4)
    
    circ.append(superPosCircuit.inverse(), [0, 1, 2])
    
    return circ

In [6]:
'''
Creates the quantum circuit
'''
qCirc = createQuantumCircuit()
qCirc.draw()

┌─────────────┐             ┌────────────────┐
q_0: ┤0            ├──────────■──┤0               ├
     │             │          │  │                │
q_1: ┤1 circuit-12 ├──────■───┼──┤1 circuit-12_dg ├
     │             │      │   │  │                │
q_2: ┤2            ├──■───┼───┼──┤2               ├
     ├─────────────┤┌─┴─┐ │   │  └────────────────┘
q_3: ┤0            ├┤ X ├─X───■────────────────────
     │             │└─┬─┘ │ ┌─┴─┐                  
q_4: ┤1            ├──■───X─┤ X ├──────────────────
     │  circuit-10 │        └───┘                  
q_5: ┤2            ├───────────────────────────────
     │             │                               
q_6: ┤3            ├───────────────────────────────
     └─────────────┘

In [8]:
'''
Pick a backend to use. We use the statevector_simulator for noiseless simulation.
'''
noiseless_backend = Aer.get_backend('statevector_simulator')
noiseless_q_instance = QuantumInstance(noiseless_backend)
noiseless_sampler = CircuitSampler(noiseless_q_instance)

In [9]:
'''
Pick a backend to use. We use the qasm_simulator for noisy simulation.
'''
#Replace the provider with your personal credentials for hub, group and main
provider = IBMQ.get_provider(hub="ibm-q-research", group="louisiana-st-uni-1", project="main")

noisy_backend = Aer.get_backend("qasm_simulator")
device = provider.get_backend("ibmq_jakarta")
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device.properties())
noisy_q_instance = QuantumInstance(backend=noisy_backend, 
                           shots=8192, 
                           noise_model=noise_model, 
                           coupling_map=coupling_map,
                           measurement_error_mitigation_cls=CompleteMeasFitter,
                           cals_matrix_refresh_period=30)
noisy_sampler = CircuitSampler(noisy_q_instance)

In [10]:
'''
Average value of this hamiltonian is the probability of measuring 0 on the first three qubit.
MatrixOp(np.array([[1, 0], [0, 0])) is the projector on the 0 subspace. Qiskit orders qubits in reverse.
'''
hamiltonian = I^I^I^I^MatrixOp(np.array([[1, 0], [0, 0]]))^MatrixOp(np.array([[1, 0], [0, 0]]))^MatrixOp(np.array([[1, 0], [0, 0]]))

In [13]:
def costf(noisy = False):
    '''
    Calculates the expectation value.
    '''
    expectation = StateFn(hamiltonian, is_measurement=True).compose(StateFn(qCirc))
    
    if noisy:
        result = noisy_sampler.convert(expectation).eval()  
    else:
        result = noiseless_sampler.convert(expectation).eval()  
    
    return np.real(result)

In [14]:
print("Cost : ", costf(False))

Cost :  0.9999999999999992
